# Proyek Analisis Data: Capital Bikeshare Bike Sharing Dataset
---
- Name: Rafli Ardiansyah
- Email: rrapliard@gmail.com
- ID Dicoding: mikachuu

## 1. Menentukan Pertanyaan Bisnis
---

1. Bagaimana tren jumlah pengguna sepeda dalam beberapa tahun terakhir?
2. Bagaimana pola penggunaan layanan bike-sharing berdasarkan waktu?
3. Pada musim apa saja jumlah pengguna layanan bike-sharing tertinggi?
4. Bagaimana pola penggunaan layanan bike-sharing berdasarkan hari dalam seminggu?
5. Apakah ada korelasi antara suhu yang mengindikasikan kondisi ketika penggunaan layanan bike-sharing tinggi?
6. Apakah cuaca mempengaruhi penggunaan bikeshare?

## Menyiapkan Library yang dibutuhkan
---

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


## Data Wrangling


### Data Gathering

In [5]:
hour_df = pd.read_csv("https://raw.githubusercontent.com/bulgogipedas/bike-sharing-analysis/main/datasets/hour.csv")
hour_df.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


- instant: indeks catatan
	- dteday : tanggal
	- season : musim (1: musim dingin, 2: musim semi, 3: musim panas, 4: musim gugur)
	- yr : tahun (0: 2011, 1: 2012)
	- bulan : bulan (1 sampai 12)
	- hr : jam (0 hingga 23)
	- holiday : hari cuaca hari libur atau tidak (diambil dari http://dchr.dc.gov/page/holiday-schedule)
	- hari kerja : hari dalam seminggu
	- hari kerja : jika hari tersebut bukan akhir pekan atau hari libur maka nilainya 1, jika tidak maka nilainya 0.
	+ cuaca : 
		- 1: Cerah, Sedikit awan, Berawan sebagian, Berawan sebagian
		- 2: Kabut + Berawan, Kabut + Awan pecah, Kabut + Sedikit awan, Kabut
		- 3: Salju Ringan, Hujan Ringan + Badai Petir + Awan berserakan, Hujan Ringan + Awan berserakan
		- 4: Hujan Lebat + Butiran Es + Badai Petir + Kabut, Salju + Kabut
	- temp: Suhu yang dinormalisasi dalam Celcius. Nilai dibagi menjadi 41 (maks)
	- atemp: Suhu perasaan yang dinormalisasi dalam Celcius. Nilai dibagi menjadi 50 (maks)
	- hum: Kelembapan yang dinormalisasi. Nilai dibagi menjadi 100 (maks)
	- windspeed: Kecepatan angin yang dinormalisasi dalam mil per jam. Nilai dibagi menjadi 67 (maks)
	- casual: jumlah perjalanan pengguna biasa
	- terdaftar: jumlah perjalanan pengguna terdaftar
	- cnt: jumlah total perjalanan sepeda sewaan termasuk kasual dan terdaftar

### 4.2 Assessing Data


Beberapa teknik dilakukan untuk memeriksa karakteristik data, misalnya memeriksa nilai yang terduplikasi, memeriksa nilai yang hilang, memeriksa nilai yang tidak akurat, dll.

In [7]:
hour_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     17379 non-null  int64  
 1   dteday      17379 non-null  object 
 2   season      17379 non-null  int64  
 3   yr          17379 non-null  int64  
 4   mnth        17379 non-null  int64  
 5   hr          17379 non-null  int64  
 6   holiday     17379 non-null  int64  
 7   weekday     17379 non-null  int64  
 8   workingday  17379 non-null  int64  
 9   weathersit  17379 non-null  int64  
 10  temp        17379 non-null  float64
 11  atemp       17379 non-null  float64
 12  hum         17379 non-null  float64
 13  windspeed   17379 non-null  float64
 14  casual      17379 non-null  int64  
 15  registered  17379 non-null  int64  
 16  cnt         17379 non-null  int64  
dtypes: float64(4), int64(12), object(1)
memory usage: 2.3+ MB


- Kita akan mengubah variabel `dteday` menjadi tipe data tanggal
- Variabel `season`, `weekday`, dan `mnth` direpresentasikan dalam bilangan bulat, kita akan mengubahnya menjadi nilai string masing-masing.
- Variabel `yr` direpresentasikan sebagai 0 untuk tahun 2011 dan 1 untuk tahun 2012. Kita akan mengubahnya menjadi nilai masing-masing.

### Check for Missing Values

In [8]:
hour_df.isna().sum()

instant       0
dteday        0
season        0
yr            0
mnth          0
hr            0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

Ternyata tidak ada nilai yang hilang dalam dataset

### Check for Duplicated Values

In [16]:
print("Jumlah duplikasi = ", end='') 
print(hour_df.duplicated().sum())

Jumlah duplikasi = 0


In [17]:
hour_df.describe(include="all")

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
count,17379.0000,17379,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000
unique,NaN,731,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,2011-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,8690.0000,NaN,2.501640,0.502561,6.537775,11.546752,0.028770,3.003683,0.682721,1.425283,0.496987,0.475775,0.627229,0.190098,35.676218,153.786869,189.463088
std,5017.0295,NaN,1.106918,0.500008,3.438776,6.914405,0.167165,2.005771,0.465431,0.639357,0.192556,0.171850,0.192930,0.122340,49.305030,151.357286,181.387599
min,1.0000,NaN,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,4345.5000,NaN,2.000000,0.000000,4.000000,6.000000,0.000000,1.000000,0.000000,1.000000,0.340000,0.333300,0.480000,0.104500,4.000000,34.000000,40.000000
50%,8690.0000,NaN,3.000000,1.000000,7.000000,12.000000,0.000000,3.000000,1.000000,1.000000,0.500000,0.484800,0.630000,0.194000,17.000000,115.000000,142.000000
75%,13034.5000,NaN,3.000000,1.000000,10.000000,18.000000,0.000000,5.000000,1.000000,2.000000,0.660000,0.621200,0.780000,0.253700,48.000000,220.000000,281.000000


In [18]:
print(hour_df.shape)

(17379, 17)


- Dataset ini terdiri dari 17.379 baris dan 17 kolom
- Seperti yang dijelaskan dalam kamus dataset, variabel `temp`, `atemp`, `hum`, dan `kecepatan angin` dinormalisasi. Untuk mendapatkan pemahaman yang lebih baik tentang kondisi kehidupan nyata, kami akan mengoreksi nilai-nilai ini sebelum dinormalisasi.

### Assessing Data Summary


- Mengubah variabel `dteday` menjadi tipe data tanggal
- Variabel `season`, `weekday`, dan `mnth` direpresentasikan dalam bilangan bulat, kita akan mengubahnya menjadi nilai string masing-masing.
- Variabel `yr` direpresentasikan sebagai 0 untuk tahun 2011 dan 1 untuk tahun 2012. Kita akan mengubahnya menjadi nilai masing-masing.
- Seperti yang dijelaskan dalam kamus dataset, variabel `temp`, `atemp`, `hum`, dan `kecepatan angin` dinormalisasi. Untuk mendapatkan pemahaman yang lebih baik tentang kondisi kehidupan nyata, kami akan mengoreksi nilai-nilai ini sebelum dinormalisasi.

### 4.3 Cleaning Data

### Convert `dteday` data type


In [20]:
hour_df['dteday'] = pd.to_datetime(hour_df['dteday'])
hour_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   instant     17379 non-null  int64         
 1   dteday      17379 non-null  datetime64[ns]
 2   season      17379 non-null  int64         
 3   yr          17379 non-null  int64         
 4   mnth        17379 non-null  int64         
 5   hr          17379 non-null  int64         
 6   holiday     17379 non-null  int64         
 7   weekday     17379 non-null  int64         
 8   workingday  17379 non-null  int64         
 9   weathersit  17379 non-null  int64         
 10  temp        17379 non-null  float64       
 11  atemp       17379 non-null  float64       
 12  hum         17379 non-null  float64       
 13  windspeed   17379 non-null  float64       
 14  casual      17379 non-null  int64         
 15  registered  17379 non-null  int64         
 16  cnt         17379 non-

In [21]:
hour_df['weekday'] = hour_df['dteday'].dt.day_name()

In [23]:
hour_df.head(2)

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,Saturday,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,Saturday,0,1,0.22,0.2727,0.80,0.0,8,32,40


In [24]:
hour_df['mnth'] = hour_df['dteday'].dt.month_name()
hour_df.head(2)

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,January,0,0,Saturday,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,January,1,0,Saturday,0,1,0.22,0.2727,0.80,0.0,8,32,40


In [25]:
# ganti nilai bilangan bulat musim dengan setiap musim yang sesuai
def find_season(season):
    season_string = {1:'Winter', 2:'Spring', 3:'Summer', 4:'Fall'}
    return season_string.get(season)

season_list = []

for season in hour_df['season']:
    season = find_season(season)
    season_list.append(season)
    
hour_df['season'] = season_list

In [27]:
hour_df.head(2)

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,Winter,0,January,0,0,Saturday,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,Winter,0,January,1,0,Saturday,0,1,0.22,0.2727,0.80,0.0,8,32,40
